In [ ]:
import tifffile
import euler_gpu
import os, random
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import numpy as np
import torch
import h5py
from tqdm import tqdm
from math import pi as PI
device = torch.device("cuda:0")
BATCH_SIZE = 256
red_channel = 1
fixed_frame = 0

MAX_NUM_CENTROIDS = 100

In [ ]:
tiff_folder = "/storage/fs/store1/brian/swimming_videos_RFa/Folder_20250212124023_RFa_noswim"
files = [f for f in os.listdir(tiff_folder) if f.endswith(".tif") or f.endswith(".tiff")]
files = sorted(files, key = lambda x: int(x.split("_")[-1].split("-")[0]))
print(files)

## If you're not using channel alignment, comment out the following line
files = ["CHAN_" + f for f in files]


['20250212_Experiment_01_0-1999.tif', '20250212_Experiment_01_2000-3999.tif', '20250212_Experiment_01_4000-5999.tif', '20250212_Experiment_01_6000-7999.tif', '20250212_Experiment_01_8000-9999.tif']


## Align Video by adjacent frames over multiple files

In [ ]:
TH_CONST = np.array([0])
XY_RANGE = np.concatenate((np.linspace(-0.005, 0.005, 10, dtype=np.float32), TH_CONST)) # Just using this here so we don't have ot redefine
ALIGN_TH_RANGE = np.concatenate((np.linspace(0, .5, 10, dtype=np.float32), np.linspace(359.5, 360, 10, dtype=np.float32), TH_CONST))

In [ ]:
tiff_path = os.path.join(tiff_folder, files[0])
inputs = tifffile.imread(tiff_path)

In [ ]:
## Align Video by adjacent frames
other_chan = (red_channel - 1) % inputs.shape[1]

fixed_image = inputs[0, red_channel]

prev_transform = (torch.tensor([0.0], device='cuda:0'), torch.tensor([0.0], device='cuda:0'), torch.tensor([0.0], device='cuda:0'))

for file in files:
    ins = tifffile.imread(os.path.join(tiff_folder, file))

    out = np.empty_like(ins)
    for frame in tqdm(range(ins.shape[0])):
    # for frame in range(red.shape[0]):
        moving_image = ins[frame, red_channel]

        align_x_range_2 = np.add(XY_RANGE, prev_transform[0].cpu().numpy())
        align_y_range_2 = np.add(XY_RANGE, prev_transform[1].cpu().numpy())
        align_the_range = np.mod(np.add(ALIGN_TH_RANGE, (prev_transform[2].cpu().numpy() * 180) / PI), 360)
        memory_dict = euler_gpu.initialize(fixed_image, moving_image, align_x_range_2, align_y_range_2, align_the_range, BATCH_SIZE, device)
        best_score, best_transformation = euler_gpu.grid_search(memory_dict)
        
        # in_img = ins[frame, red_channel]
        in_img = torch.Tensor(moving_image[np.newaxis, np.newaxis, ...]).to(device=device)
        out[frame, red_channel] = euler_gpu.transform_image(in_img, best_transformation[0], best_transformation[1], best_transformation[2], memory_dict).cpu().numpy()
        in_img = ins[frame, other_chan]
        in_img = torch.Tensor(in_img[np.newaxis, np.newaxis, ...]).to(device=device)
        out[frame, other_chan] = euler_gpu.transform_image(in_img, best_transformation[0], best_transformation[1], best_transformation[2], memory_dict).cpu().numpy()
        # print(best_transformation)

        prev_transform = best_transformation
    print(prev_transform)
    tifffile.imwrite(os.path.join(tiff_folder, "RIG_" + file), out, imagej=True, bigtiff=True)



100%|██████████| 2000/2000 [40:43<00:00,  1.22s/it]


(tensor([0.0100], device='cuda:0'), tensor([-0.0111], device='cuda:0'), tensor([0.0019], device='cuda:0'))


100%|██████████| 2000/2000 [41:25<00:00,  1.24s/it]


(tensor([0.0156], device='cuda:0'), tensor([-0.0211], device='cuda:0'), tensor([0.0019], device='cuda:0'))


/home/brian/.conda/envs/PB/lib/python3.11/site-packages/tifffile/tifffile.py:3784: UserWarning: <tifffile.TiffWriter 'RIG_CHAN_20240930_2000-3999.tif'> truncating ImageJ file
  warnings.warn(
100%|██████████| 2000/2000 [40:38<00:00,  1.22s/it]


...

/home/brian/.conda/envs/PB/lib/python3.11/site-packages/tifffile/tifffile.py:3784: UserWarning: <tifffile.TiffWriter 'RIG_CHAN_202409…_24000-25999.tif'> truncating ImageJ file
  warnings.warn(
100%|██████████| 2000/2000 [41:51<00:00,  1.26s/it]


(tensor([0.0989], device='cuda:0'), tensor([-0.1233], device='cuda:0'), tensor([6.2726], device='cuda:0'))


/home/brian/.conda/envs/PB/lib/python3.11/site-packages/tifffile/tifffile.py:3784: UserWarning: <tifffile.TiffWriter 'RIG_CHAN_202409…_26000-27999.tif'> truncating ImageJ file
  warnings.warn(
100%|██████████| 2000/2000 [40:48<00:00,  1.22s/it]


(tensor([0.0989], device='cuda:0'), tensor([-0.1233], device='cuda:0'), tensor([6.2726], device='cuda:0'))


/home/brian/.conda/envs/PB/lib/python3.11/site-packages/tifffile/tifffile.py:3784: UserWarning: <tifffile.TiffWriter 'RIG_CHAN_202409…_28000-29999.tif'> truncating ImageJ file
  warnings.warn(
